In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp
import h5py

plt.rc('figure', figsize=(6,4))
plt.rc('font', size=11)
plt.style.use('dark_background')
# plt.rc('figure',facecolor=(0,0,0,0)) # Set transparent background
%config InlineBackend.figure_format='retina'

In [23]:
import quflow as qf

#qf.gpu.check_status()

In [24]:
N = 200  # Size of matrices
lmax = 10  # How many spherical harmonics (SH) coefficients to include
np.random.seed(42)  # For reproducability
omega0 = np.random.randn(lmax**2)  # Array with SH coefficients
W0 = qf.shr2mat(omega0, N=N)  # Convert SH coefficients to matrix

filename_cpu = "cpu_test_sim_N_{}.hdf5".format(str(N))
filename_gpu = "gpu_test_sim_N_{}.hdf5".format(str(N))
time = 3.0 # in second
inner_time = 0.5 # in seconds
qstepsize = 0.2 # in qtime

# Run this if you want to remove the existing hdf5 file
%rm $filename_cpu
%rm $filename_gpu

dt = qf.qtime2seconds(qstepsize, N)
print("The physical stepsize is {:.3e} seconds.".format(dt))
print("{} steps per output, in total {} steps.".format(round(inner_time/dt), round(time/dt)))

The physical stepsize is 5.013e-04 seconds.
997 steps per output, in total 5984 steps.


In [9]:
"""
CPU 
"""
# Callback data object
mysim_cpu = qf.QuData(filename_cpu)

# Save initial conditions if file does not exist already, otherwise load from last step
try:
    f = h5py.File(filename_cpu, "r")
except IOError or KeyError:
    W = W0.copy()
    mysim_cpu(W, 0.0)
else:
    W = qf.shr2mat(f['state'][-1,:], N=N)
    assert W.shape[0] == N, "Looks like the saved data use N = {} whereas you specified N = {}.".format(W.shape[0], N)
    f.close()

# Select solver
method = qf.isomp_fixedpoint
method_kwargs = {"hamiltonian": qf.solve_poisson, "verbatim":False, "maxit":7, "tol":1e-8}

# Run simulation

qf.solve(W, qstepsize=qstepsize, time=time, inner_time=inner_time, callback=mysim_cpu,
         method=method, method_kwargs=method_kwargs)

# Flush cache data
mysim_cpu.flush()

/home/filip/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 5984/5984 [07:31<00:00, 13.25 steps/s]


In [25]:

"""
GPU
"""
# Initialize solver
isomp_solver = qf.gpu.isomp_gpu_skewherm_solver(W0)

# Callback data object
mysim_gpu = qf.QuData(filename_gpu)

# Save initial conditions if file does not exist already, otherwise load from last step
try:
    f = h5py.File(filename_gpu, "r")
except IOError or KeyError:
    W = W0.copy()
    mysim_gpu(W, 0.0)
else:
    W = qf.shr2mat(f['state'][-1,:], N=N)
    assert W.shape[0] == N, "Looks like the saved data use N = {} whereas you specified N = {}.".format(W.shape[0], N)
    f.close()

# Select solver
# The cupy based isomp solver and poisson solver needs to be initialized 
method = qf.gpu.isomp_gpu_skewherm_solver(W0)
ham = qf.gpu.solve_poisson_interleaved_cp(N)

# Method kwargs are the same, since we still use the same qf.solve from dynamics.py
method_kwargs = {"hamiltonian": ham.solve_poisson, "verbatim":False, "maxit":7, "tol":1e-8}

# Run simulation
# Run simulation


qf.solve(W, qstepsize=qstepsize, time=time, inner_time=inner_time, callback=mysim_gpu,
         method=method.solve_step, method_kwargs=method_kwargs)

# Flush cache data
mysim_gpu.flush()

100%|██████████| 5984/5984 [01:26<00:00, 69.29 steps/s]


In [26]:
# Plot last state
with h5py.File(filename_gpu, 'r') as data:
    omega = data['state'][-1]
    qf.plot2(omega, projection='hammer', N=520, colorbar=True)
    #plt.savefig(f"output_{N}.png")

In [27]:
with h5py.File(filename_gpu, 'r') as data:
    anim = qf.create_animation2(filename_gpu.replace(".hdf5",".mp4"), data['state'], projection='mollweide', N=256)
anim

100%|██████████| 8/8 [00:02<00:00,  3.21 frames/s]


## Compare GPU solution to CPU

In [28]:
with h5py.File(filename_gpu, 'r') as data_gpu, h5py.File(filename_cpu, 'r') as data_cpu:
    diff = np.zeros(data_gpu['state'].shape[0])
    for k in range(data_gpu['state'].shape[0]):
        diff[k] = np.linalg.norm(data_cpu['state'][k]-data_gpu['state'][k])
    plt.figure(figsize=(8,4))
    plt.plot(diff)
    print(diff)
    plt.show()



[0.00000000e+00 4.64833935e-10 1.02046373e-09 1.84531035e-09
 3.20227302e-09 5.26456507e-09 7.99939259e-09 8.00544788e-09]


## Check conservation of casimirs


In [29]:
# Check conservation of casimirs

casimir_gpu = []
casimir_cpu = []
with h5py.File(filename_gpu, 'r') as data:
    print(data['state'].shape[0])
    for k in range(data['state'].shape[0]):
        omega = data['state'][k]
        casimir_gpu += [np.linalg.norm(omega)**2]
    casimir_gpu = np.asarray(casimir_gpu)
    plt.figure(figsize=(8,4))
    plt.plot(casimir_gpu, label="gpu")

with h5py.File(filename_cpu, 'r') as data:
    print(data['state'].shape[0])
    for k in range(data['state'].shape[0]): 
        omega = data['state'][k] # type: ignore
        casimir_cpu += [np.linalg.norm(omega)**2]
    casimir_cpu = np.asarray(casimir_cpu)
    plt.plot(casimir_cpu, label="cpu")

plt.legend()
plt.show()
print(np.abs(casimir_gpu-casimir_cpu))

8
8
[0.00000000e+00 0.00000000e+00 0.00000000e+00 9.94759830e-14
 2.84217094e-14 2.84217094e-14 2.84217094e-14 4.26325641e-14]
